In [1]:
pip install python-docx pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.0 MB/s eta 0:00:00


In [13]:
import pandas as pd
import docx
from datetime import datetime
import re
import numpy as np

# --- Configuration ---
# The script will read all content from these three files.
# Ensure they are in the same directory as the script.
CONTENT_DOC_PATH = 'daily_content.docx'
FEE_TABLE_XLSX_PATH = '/content/fee_table.xlsx'
SQUEEZE_TABLE_XLSX_PATH = '/content/squeeze_table.xlsx' # Updated path to the Excel file
OUTPUT_HTML_PATH = f"equilend_daily_digest_{datetime.now().strftime('%Y_%m_%d')}.html"

# --- 1. DOCX Content Parsing ---

def parse_daily_content(doc_path):
    """
    Parses the .docx file for all text content, including market notes, headlines, and key takeaways.
    """
    try:
        document = docx.Document(doc_path)
    except Exception as e:
        print(f"Error opening or parsing DOCX file: {e}")
        return None

    content = {'market_notes': {}, 'headlines': [], 'key_takeaways': []}
    paras = [p.text.strip() for p in document.paragraphs if p.text.strip()]

    # --- Parse Market Notes ---
    try:
        market_notes_idx = -1
        for i, p_text in enumerate(paras):
            if 'market notes' in p_text.lower():
                market_notes_idx = i
                break

        if market_notes_idx != -1:
            for i in range(market_notes_idx + 1, len(paras)):
                line = paras[i]
                if not (line.startswith('*') or line.startswith('•')): break

                line = line.lstrip('*• ').strip()
                if 'Average Fee:' in line:
                    match = re.search(r'(\d+\.?\d*)\s*bps\s*\((.*)\)', line)
                    if not match: # Fallback for different format
                        match = re.search(r'(\d+\.?\d*)\s*bps\s*(.*)', line)
                    if match:
                        content['market_notes']['avg_fee_bps'] = match.group(1)
                        content['market_notes']['avg_fee_change'] = float(match.group(2).replace('%','').replace('+',''))
                elif 'Average Utilization:' in line:
                    match = re.search(r'(\d+\.?\d*)%\s*\((.*)\)', line)
                    if not match:
                         match = re.search(r'(\d+\.?\d*)%\s*(.*)', line)
                    if match:
                        content['market_notes']['avg_util_percent'] = match.group(1)
                        content['market_notes']['avg_util_change'] = float(match.group(2).replace('%','').replace('+',''))
                elif 'On Loan Value:' in line:
                    match = re.search(r'\$(\d+\.?\d*T)\s*\((.*)\)', line)
                    if not match:
                        match = re.search(r'(\d+\.?\d*T)\s*(.*)', line)
                    if match:
                        content['market_notes']['on_loan_value'] = match.group(1)
                        content['market_notes']['on_loan_value_change'] = float(match.group(2).replace('%','').replace('+',''))
                elif 'Total Lendable Value:' in line:
                    match = re.search(r'\$(\d+\.?\d*T)\s*\((.*)\)', line)
                    if not match:
                        match = re.search(r'(\d+\.?\d*T)\s*(.*)', line)
                    if match:
                        content['market_notes']['lendable_value'] = match.group(1)
                        content['market_notes']['lendable_value_change'] = float(match.group(2).replace('%','').replace('+',''))
    except Exception as e:
        print(f"Warning: Could not parse Market Notes. Error: {e}")

    # --- Parse Headlines ---
    try:
        headline_start_index = -1
        for i, p_text in enumerate(paras):
            if 'major headlines' in p_text.lower():
                headline_start_index = i + 1
                break

        if headline_start_index != -1:
            i = headline_start_index
            while i < len(paras):
                end_of_block_idx = -1
                # Find the next data insight line
                for j in range(i + 1, len(paras)):
                    if 'what our data shows' in paras[j].lower():
                        end_of_block_idx = j
                        break

                if end_of_block_idx != -1:
                    title = paras[i].rstrip(':').rstrip('.')
                    description = "\n".join(paras[i+1:end_of_block_idx])
                    data_insight_raw = paras[end_of_block_idx]
                    data_insight = re.sub(r'[\starbullet\s]*What our data shows:\s?', '', data_insight_raw, flags=re.IGNORECASE).strip()

                    content['headlines'].append({
                        'title': title,
                        'description': description,
                        'data_insight': data_insight
                    })
                    i = end_of_block_idx + 1
                else:
                    break # No more headline blocks found
    except Exception as e:
        print(f"Warning: Could not parse Headlines. Error: {e}")

    # --- Parse Key Takeaways ---
    try:
        takeaways_start_index = -1
        for i, p_text in enumerate(paras):
            if 'key takeaways' in p_text.lower():
                takeaways_start_index = i + 1
                break
        if takeaways_start_index != -1:
            for i in range(takeaways_start_index, len(paras)):
                line = paras[i].lstrip('-•* ').strip()
                if not line: continue
                content['key_takeaways'].append(line)
    except Exception as e:
        print(f"Warning: Could not parse Key Takeaways. Error: {e}")

    return content

# --- 2. Table Parsing ---

def parse_tables(fee_path, squeeze_path):
    """
    Parses the fee table from an Excel file and the squeeze table from an Excel file.
    """
    ht_borrow_html = ""
    squeeze_html = ""

    def format_change(value, unit=' BPS'):
        try:
            cleaned_value = str(value).replace(',', '').strip()
            val = float(cleaned_value)
            color = '#16a34a' if val > 0 else '#dc2626'
            sign = '+' if val > 0 else ''
            return f'<span style="color:{color}; font-size:14px;">{sign}{val:,.2f}{unit}</span>'
        except (ValueError, TypeError):
            return str(value).strip() if pd.notna(value) else ''

    # --- Parse "Hot & Getting Hotter" Table from Excel ---
    try:
        fee_df = pd.read_excel(fee_path) # Read Excel file
        # Dynamically find the WoW column (could be 'Fee WoW (bps)' or 'Fee 1 Week Diff (Bps)', etc.)
        wow_col_fee = next((col for col in fee_df.columns if 'WoW' in col and 'Fee' in col), None)
        price_col_fee = next((col for col in fee_df.columns if 'Price' in col), None)

        for _, row in fee_df.iterrows():
            ht_borrow_html += f"""
            <tr style="font-size:12px; background-color:#ffffff; text-align:left;">
                <td style="padding:4px 8px;">{row.get("TICKER", "")}</td>
                <td style="padding:4px 8px;">{row.get("Security Description", "")}</td>
                <td style="padding:4px 8px;">{row.get("Industry", "")}</td>
                <td style="padding:4px 8px;">{row.get(price_col_fee, "")}</td>
                <td style="padding:4px 8px;">{format_change(row.get(wow_col_fee))}</td>
            </tr>"""
    except FileNotFoundError:
        print(f"ERROR: Fee Table Excel not found at '{fee_path}'. Please provide the file.")
    except Exception as e:
        print(f"Warning: Could not parse Fee Table Excel. Error: {e}")


    # --- Parse "Squeeze Metrics" Table from Excel ---
    try:
        squeeze_df = pd.read_excel(squeeze_path)
        wow_col_squeeze = next((col for col in squeeze_df.columns if 'WoW' in col and 'Squeeze' in col), None)
        price_col_squeeze = next((col for col in squeeze_df.columns if 'Price' in col), None)


        for _, row in squeeze_df.iterrows():
            squeeze_html += f"""
            <tr style="font-size:12px; background-color:#ffffff; text-align:left;">
                <td style="padding:4px 8px;">{row.get("TICKER", "")}</td>
                <td style="padding:4px 8px;">{row.get("Security Description", "")}</td>
                <td style="padding:4px 8px;">{row.get("Industry", "")}</td>
                <td style="padding:4px 8px;">{row.get(price_col_squeeze, "")}</td>
                <td style="padding:4px 8px;">{row.get("Squeeze Score", "")}</td>
                <td style="padding:4px 8px;">{format_change(row.get(wow_col_squeeze), unit="")}</td>
            </tr>"""
    except FileNotFoundError:
        print(f"ERROR: Squeeze Table Excel not found at '{squeeze_path}'. Please provide the file.")
    except Exception as e:
        print(f"Warning: Could not parse Squeeze Table Excel. Error: {e}")

    return ht_borrow_html, squeeze_html

# --- 3. HTML Generation ---

def create_digest_html(content, table_html_tuple):
    """
    Generates the final HTML for the digest by populating a template.
    """
    if not content:
        return "<h1>Error: Could not generate content.</h1>"

    ht_borrow_table, squeeze_table = table_html_tuple

    def format_market_note_change(value):
        color = '#16a34a' if value >= 0 else '#dc2626'
        sign = '+' if value >= 0 else ''
        return f'<span style="color:{color}; font-size:14px;">{sign}{value:.2f}%</span>'

    market_notes = content.get('market_notes', {})
    avg_fee_bps = market_notes.get('avg_fee_bps', 'N/A')
    avg_fee_change = format_market_note_change(market_notes.get('avg_fee_change', 0))
    avg_util_percent = market_notes.get('avg_util_percent', 'N/A')
    avg_util_change = format_market_note_change(market_notes.get('avg_util_change', 0))
    on_loan_value = market_notes.get('on_loan_value', 'N/A')
    on_loan_value_change = format_market_note_change(market_notes.get('on_loan_value_change', 0))
    lendable_value = market_notes.get('lendable_value', 'N/A')
    lendable_value_change = format_market_note_change(market_notes.get('lendable_value_change', 0))

    headlines_html = ""
    for headline in content.get('headlines', []):
        headlines_html += f"""
        <div style="margin-top:20px;">
            <strong style="font-size:16px;">{headline['title']}</strong>
            <p style="margin:5px 0 0; white-space: pre-wrap;">{headline['description']}</p>
            <ul style="margin:10px 0 0; padding-left:20px; color:#334155; list-style-type: disc;">
              <li><strong>What Our Data Shows:</strong> {headline['data_insight']}</li>
            </ul>
        </div>
        """

    takeaways_html = "".join([f"<li>{item}</li>" for item in content.get('key_takeaways', [])])

    html_template = f"""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>EquiLend D&A Daily Digest - {datetime.now().strftime('%B %d, %Y')}</title>
</head>
<body style="margin:0; padding:0; font-family: Arial, sans-serif; background-color:#f1f5f9;">
  <table width="100%" cellpadding="0" cellspacing="0" border="0" bgcolor="#f1f5f9">
    <tr>
      <td align="center">
        <table width="800" cellpadding="0" cellspacing="0" border="0" bgcolor="#ffffff" style="margin:20px 0; border-radius:4px; overflow:hidden;">
          <!-- Header -->
          <tr>
            <td bgcolor="#0284c7" style="padding:20px; text-align:center;">
              <h1 style="margin:0; font-size:24px; color:#ffffff;">EquiLend D&amp;A Daily Digest</h1>
              <p style="margin:5px 0 0; font-size:16px; color:#bae6fd;">{datetime.now().strftime('%B %d, %Y')}</p>
            </td>
          </tr>
          <!-- Content -->
          <tr>
            <td style="padding:20px; color:#334155; font-size:14px; line-height:1.5;">

              <!-- Market Notes -->
              <h2 style="font-size:18px; margin-bottom:10px; color:#0284c7;">&#x1F4BC; Market Notes</h2>
              <table width="100%" cellpadding="5" cellspacing="0" border="0" bgcolor="#f8fafc" style="margin-bottom:20px;">
                <tr>
                  <td align="center" width="50%">
                    <p style="margin:0; font-size:12px; color:#64748b;">Average Fee</p>
                    <p style="margin:5px 0 0; font-size:18px; font-weight:bold; color:#334155;">
                      {avg_fee_bps} bps {avg_fee_change}
                    </p>
                  </td>
                  <td align="center" width="50%">
                    <p style="margin:0; font-size:12px; color:#64748b;">Average Utilization</p>
                    <p style="margin:5px 0 0; font-size:18px; font-weight:bold; color:#334155;">
                      {avg_util_percent}% {avg_util_change}
                    </p>
                  </td>
                </tr>
                <tr>
                  <td align="center" width="50%">
                    <p style="margin:0; font-size:12px; color:#64748b;">On Loan Value</p>
                    <p style="margin:5px 0 0; font-size:18px; font-weight:bold; color:#334155;">
                      ${on_loan_value} {on_loan_value_change}
                    </p>
                  </td>
                  <td align="center" width="50%">
                    <p style="margin:0; font-size:12px; color:#64748b;">Total Lendable Value</p>
                    <p style="margin:5px 0 0; font-size:18px; font-weight:bold; color:#334155;">
                      ${lendable_value} {lendable_value_change}
                    </p>
                  </td>
                </tr>
              </table>

              <!-- Headlines -->
              <h2 style="font-size:18px; margin-bottom:10px; color:#0284c7;">&#x1F4F0; Major Headlines and What Our Data Shows</h2>
              {headlines_html}

              <!-- Tables Section -->
              <h2 style="font-size:18px; margin:20px 0 10px; color:#0284c7;">&#x1F525; Hot & Getting Hotter</h2>
              <div style="font-size:14px; font-style:italic; color:#334155; margin-bottom:10px; padding-left:5px;">Top 5 Stocks with fees > 50,000 bps & rising</div>
              <table cellpadding="0" cellspacing="0" border="1" style="border-collapse:collapse; margin-bottom:20px; border-color:#dddddd; font-size:12px;" width="100%">
                <thead>
                  <tr bgcolor="#0284c7" style="color:#ffffff; font-weight:bold; text-align:left;">
                    <th style="padding:5px 8px; white-space: nowrap;">Ticker</th>
                    <th style="padding:5px 8px; white-space: nowrap;">Company Name</th>
                    <th style="padding:5px 8px; white-space: nowrap;">Industry</th>
                    <th style="padding:5px 8px; white-space: nowrap;">Price</th>
                    <th style="padding:5px 8px; white-space: nowrap;">Fee 1 WoW (bps)</th>
                  </tr>
                </thead>
                <tbody>
                  {ht_borrow_table}
                </tbody>
              </table>

              <h2 style="font-size:18px; margin:20px 0 10px; color:#0284c7;">&#x1F52C; Market Technicals & Squeeze Metrics</h2>
              <table cellpadding="0" cellspacing="0" border="1" style="border-collapse:collapse; border-color:#dddddd; font-size:12px;" width="100%">
                <thead>
                  <tr bgcolor="#0284c7" style="color:#ffffff; font-weight:bold; text-align:left;">
                    <th style="padding:5px 8px; white-space: nowrap;">Ticker</th>
                    <th style="padding:5px 8px; white-space: nowrap;">Company Name</th>
                    <th style="padding:5px 8px; white-space: nowrap;">Industry</th>
                    <th style="padding:5px 8px; white-space: nowrap;">Price</th>
                    <th style="padding:5px 8px; white-space: nowrap;">Short Squeeze Score (SSS)</th>
                    <th style="padding:5px 8px; white-space: nowrap;">SSS WoW</th>
                  </tr>
                </thead>
                <tbody>
                  {squeeze_table}
                </tbody>
              </table>

              <!-- Key Takeaways -->
              <h2 style="font-size:18px; margin:20px 0 10px; color:#0284c7;">&#x1F4A1; Key Takeaways</h2>
              <ul style="padding-left:20px; color:#334155; font-size:14px; margin:0;">
                {takeaways_html}
              </ul>

            </td>
          </tr>
          <!-- Footer -->
          <tr>
            <td align="center" style="padding:10px; font-size:12px; color:#64748b;">
              Source: EquiLend Data &amp; Analytics
            </td>
          </tr>
        </table>
      </td>
    </tr>
  </table>
</body>
</html>
    """
    return html_template

# --- Main Execution ---
if __name__ == "__main__":
    print("Starting daily digest generation...")

    # 1. Parse content from the Word document
    print(f"Reading content from '{CONTENT_DOC_PATH}'...")
    parsed_content = parse_daily_content(CONTENT_DOC_PATH)

    # 2. Parse data from the table files
    print(f"Reading tables from '{FEE_TABLE_XLSX_PATH}' and '{SQUEEZE_TABLE_XLSX_PATH}'...") # Updated print message
    table_html = parse_tables(FEE_TABLE_XLSX_PATH, SQUEEZE_TABLE_XLSX_PATH) # Call the updated function

    # 3. Generate the final HTML file
    print("Generating final HTML digest...")
    final_html = create_digest_html(parsed_content, table_html)

    # 4. Save the HTML to a file
    with open(OUTPUT_HTML_PATH, 'w', encoding='utf-8') as f:
        f.write(final_html)

    print(f"\nSuccessfully generated digest: {OUTPUT_HTML_PATH}")

Starting daily digest generation...
Reading content from 'daily_content.docx'...
Reading tables from '/content/fee_table.xlsx' and '/content/squeeze_table.xlsx'...
Generating final HTML digest...

Successfully generated digest: equilend_daily_digest_2025_08_06.html
